## VGG
VGGNet采用了五组卷积与三个全连接层，最后使用Softmax做分类。VGGNet有一个显著的特点：每次经过池化层（maxpool）后特征图的尺寸减小一倍，而通道数则增加一倍（最后一个池化层除外）。

具体为：

卷积-卷积-池化-卷积-卷积-池化-卷积-卷积-卷积-池化-卷积-卷积-卷积-池化-卷积-卷积-卷积-池化-全连接-全连接-全连接 。

通道数分别为64，128，512，512，512，4096，4096，1000。卷积层通道数翻倍，直到512时不再增加。通道数的增加，使更多的信息被提取出来。全连接的4096是经验值，当然也可以是别的数，但是不要小于最后的类别。1000表示要分类的类别数。

所有的激活单元都是Relu 。

用池化层作为分界，VGG16共有6个块结构，每个块结构中的通道数相同。因为卷积层和全连接层都有权重系数，也被称为权重层，其中卷积层13层，全连接3层，池化层不涉及权重。所以共有13+3=16权重层。

对于VGG16卷积神经网络而言，其13层卷积层和5层池化层负责进行特征的提取，最后的3层全连接层负责完成分类任务。

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import numpy as np

"""### 建立 block"""

class Block(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, padding=1, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.relu1 = nn.ReLU(inplace=True)
    def forward(self, x):
        out = self.relu1(self.bn1(self.conv1(x)))
        return out

"""### 建立 layer"""

def make_layers(in_channels, layer_list):
# 使用 make_layers 函数，来自动创建 Sequential 网络层列表。
    layers = []
    for out_channels in layer_list:
      layers += [Block(in_channels, out_channels)]
      in_channels = out_channels
    return nn.Sequential(*layers)


class Layer(nn.Module):
    def __init__(self, in_channels, layer_list):
        super(Layer, self).__init__()
        self.layer = make_layers(in_channels, layer_list)
    def forward(self, x):
        out = self.layer(x)
        return out

"""
建立 VGG 模型
D 和 E 两种最为常用，即我们所说的 VGG16 和 VGG19
数字代表通道数，如 64 表示输出 64 通道特征图，对应于论文中的 Conv3-64;
M 代表最大池化操作，对应于论文中的 maxpool 
* 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
* 'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
* 'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
* 'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
"
"""

class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, n_class=21):
        super(VGG, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # 这里 stride 默认=1
        # (s-3+2)/1 + 1 = s
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.layer1 = Layer(64, [64]) # in_channels = 64, layer_list = [64]
        # make_layers() -> Block conv-bn-relu channel=64
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer2 = Layer(64, [128, 128])
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer3 = Layer(128, [256, 256, 256, 256])
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer4 = Layer(256, [512, 512, 512, 512])
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer5 = Layer(512, [512, 512, 512, 512])
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        # VGGNet 的 3 个全连接层，中间有 ReLU 与 Dropout
        self.fc6 = nn.Linear(512*7*7, 4096)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout()

        self.fc7 = nn.Linear(4096, 4096)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout()

        self.score = nn.Linear(4096, n_class)

    def forward(self, x):
        f0 = self.relu1(self.bn1(self.conv1(x)))
        f1 = self.pool1(self.layer1(f0))
        f2 = self.pool2(self.layer2(f1))
        f3 = self.pool3(self.layer3(f2))
        f4 = self.pool4(self.layer4(f3))
        f5 = self.pool5(self.layer5(f4))
        f5 = f5.view(f5.size(0), -1)
        print(f5.shape)
        f6 = self.drop6(self.relu6(self.fc6(f5)))
        f7 = self.drop7(self.relu7(self.fc7(f6)))
        score = self.score(f7)
        return score

vgg_model = VGG()
x = torch.randn((2, 3, 224, 224), dtype=torch.float32)
y = vgg_model(x)
print(y.shape)